In [257]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

In [264]:
df = pd.read_csv('C:\\Users\\binary\\Downloads\\data_set.csv',encoding = 'latin') 

In [265]:
#exploratory data analysis

print(df.shape)
df.head()

(40027, 20)


,APPLICANT_ID,LEAVE_TYPE,STAFF_NO,DESIGNATION,LOCATION,HEAD_OF_DEPARTMENT,BUSINESS,DEPARTMENT,SUPERVISOR,S_DATE,S_Month,E_DATE,SEASON,TOTAL_NO_DAYS,IS_ACTIVE,REASON,LEAVE_STATUS_ID,BUSINESS_MANAGER,LEAVE_CODE,STATUS
0,102897,PL,159,Accounts Officer,Abbbottabad,Others,Business 5,Others,Others,4-Dec-15,12,12/5/2015,1,1,1.0,1,8,Syed Akram,9001,Inprogress
1,105771,PL,159,Accounts Officer,Abbbottabad,Others,Business 5,Others,Others,17-Dec-16,12,12/17/2016,1,0,1.0,test,3,Shabbir Ali,9004,Posted
2,102974,PL,221,Accounts Officer,Abbbottabad,Others,Business 5,Cirin Pharma,Others,29-Sep-16,9,9/30/2016,4,1,1.0,test,8,Shabbir Ali,9001,Posted
3,102978,PL,221,Accounts Officer,Abbbottabad,Others,Business 5,Cirin Pharma,Others,30-Sep-16,9,10/1/2016,4,1,1.0,test 02,3,Shabbir Ali,9004,Posted
4,102982,SL,221,Accounts Officer,Abbbottabad,Others,Business 5,Cirin Pharma,Others,14-Oct-16,10,10/18/2016,4,4,1.0,test,8,Umran Shahid,9001,Posted


In [266]:
#filtering out data that is approved to predict the number of leaves that will be approved

df = df[df['STATUS']=='Posted']
print(df.shape)
df.head()

(26828, 20)


,APPLICANT_ID,LEAVE_TYPE,STAFF_NO,DESIGNATION,LOCATION,HEAD_OF_DEPARTMENT,BUSINESS,DEPARTMENT,SUPERVISOR,S_DATE,S_Month,E_DATE,SEASON,TOTAL_NO_DAYS,IS_ACTIVE,REASON,LEAVE_STATUS_ID,BUSINESS_MANAGER,LEAVE_CODE,STATUS
1,105771,PL,159,Accounts Officer,Abbbottabad,Others,Business 5,Others,Others,17-Dec-16,12,12/17/2016,1,0,1.0,test,3,Shabbir Ali,9004,Posted
2,102974,PL,221,Accounts Officer,Abbbottabad,Others,Business 5,Cirin Pharma,Others,29-Sep-16,9,9/30/2016,4,1,1.0,test,8,Shabbir Ali,9001,Posted
3,102978,PL,221,Accounts Officer,Abbbottabad,Others,Business 5,Cirin Pharma,Others,30-Sep-16,9,10/1/2016,4,1,1.0,test 02,3,Shabbir Ali,9004,Posted
4,102982,SL,221,Accounts Officer,Abbbottabad,Others,Business 5,Cirin Pharma,Others,14-Oct-16,10,10/18/2016,4,4,1.0,test,8,Umran Shahid,9001,Posted
5,103000,SL,221,Accounts Officer,Abbbottabad,Others,Business 5,Cirin Pharma,Others,29-Sep-16,9,10/6/2016,4,7,1.0,testing,8,Syed Akram,9001,Posted


In [267]:
df.keys()

Index(['APPLICANT_ID', 'LEAVE_TYPE', 'STAFF_NO', 'DESIGNATION', 'LOCATION',
       'HEAD_OF_DEPARTMENT', 'BUSINESS', 'DEPARTMENT', 'SUPERVISOR', 'S_DATE',
       'S_Month', 'E_DATE', 'SEASON', 'TOTAL_NO_DAYS', 'IS_ACTIVE', 'REASON',
       'LEAVE_STATUS_ID', 'BUSINESS_MANAGER', 'LEAVE_CODE', 'STATUS'],
      dtype='object')

In [268]:
df.dtypes

APPLICANT_ID            int64
LEAVE_TYPE             object
STAFF_NO               object
DESIGNATION            object
LOCATION               object
HEAD_OF_DEPARTMENT     object
BUSINESS               object
DEPARTMENT             object
SUPERVISOR             object
S_DATE                 object
S_Month                 int64
E_DATE                 object
SEASON                  int64
TOTAL_NO_DAYS           int64
IS_ACTIVE             float64
REASON                 object
LEAVE_STATUS_ID         int64
BUSINESS_MANAGER       object
LEAVE_CODE              int64
STATUS                 object
dtype: object

In [269]:
# feature elimination: discarding the irrelevant fields that won't have an effect on the number of leaves availed and on the overall predictions. 
df = df.drop(['APPLICANT_ID','IS_ACTIVE','REASON','STATUS'], axis = 1)

In [270]:
#spliting the data on training and testing
# data recorded in the year 2018 is used for testing and the model will be trained on the data before that

test = df[pd.to_datetime(df['S_DATE']).dt.year == 2018]
train = df[pd.to_datetime(df['S_DATE']).dt.year != 2018]

In [271]:
df['LEAVE_TYPE'].value_counts()

PL     17770
SL      5567
LW      1620
EPL     1127
LR       341
PAL      221
OL       120
MAL       29
TL        28
PTL        5
Name: LEAVE_TYPE, dtype: int64

In [113]:
df['LEAVE_CODE'].value_counts()

9001    26555
9004     8047
9010     2218
9012     1631
1111      509
9011      475
9013      318
9019      180
9003       52
9005       35
9002        7
Name: LEAVE_CODE, dtype: int64

In [272]:
# sdate and edate was helpful just for spliting data and won't contribute to the overall predictions so eliminating them as well 

train = train.drop(['S_DATE','E_DATE'], axis = 1)
test = test.drop(['S_DATE','E_DATE'], axis = 1)

In [273]:
print(train.shape)
print(test.shape)

(26381, 14)
(447, 14)


In [274]:
df.keys()


Index(['LEAVE_TYPE', 'STAFF_NO', 'DESIGNATION', 'LOCATION',
       'HEAD_OF_DEPARTMENT', 'BUSINESS', 'DEPARTMENT', 'SUPERVISOR', 'S_DATE',
       'S_Month', 'E_DATE', 'SEASON', 'TOTAL_NO_DAYS', 'LEAVE_STATUS_ID',
       'BUSINESS_MANAGER', 'LEAVE_CODE'],
      dtype='object')

In [275]:
#assigning the features and targets variable. 

X_train = train.drop(['TOTAL_NO_DAYS'],axis = 1)
Y_train = train['TOTAL_NO_DAYS']

X_test = test.drop(['TOTAL_NO_DAYS'],axis = 1)
Y_test = test['TOTAL_NO_DAYS']

In [276]:
# combining the train and test features to label encode. there are a lot of new features that are present in 
# the testing data but not the training data like a new depatment added later in the years or any new employee joining.
# to avoid label encoder throwing an error we combine the data and fit that
data = X_train.append(X_test)

In [277]:
data.shape

(26828, 13)

In [278]:
#label encoder is used to encode features in a format that can  be understood by the model
# we fit the label encoder on the entire dataset and later on use it to transform our training and test sets

le = LabelEncoder()
for i in range(0,12):
    data.iloc[:,i] = le.fit_transform(data.iloc[:,i])
    X_test.iloc[:,i] = le.transform(X_test.iloc[:,i])
    X_train.iloc[:,i] = le.transform(X_train.iloc[:,i])

In [279]:
# since the spread of the data is a bit skewed and does not follow any fixed pregression non linear models like
# tree based algorithms can woek best. random forest being the most popular choice we go for this 
# keeping the samples split to 20 to bring about a generalizaton and avoid overfiting 
model = RandomForestRegressor(min_samples_split = 20,n_jobs = -1)

In [280]:
model.fit(X_train.values,Y_train.values)

c:\python36\scripts\osamaenv\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=20,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [281]:
# model predicting test values
pred = model.predict(X_test)

In [282]:
#changing format to integer
pred = pred.astype(int)

In [283]:
#error on a single prediciton
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y_test,pred)

2.335570469798658

In [284]:
#appending predictions to test data
test['pred'] = pred

In [285]:
test

,LEAVE_TYPE,STAFF_NO,DESIGNATION,LOCATION,HEAD_OF_DEPARTMENT,BUSINESS,DEPARTMENT,SUPERVISOR,S_Month,SEASON,TOTAL_NO_DAYS,LEAVE_STATUS_ID,BUSINESS_MANAGER,LEAVE_CODE,pred
127,PL,2197,Accounts Officer,KHI,Others,Business 5,Shared Services Accounting,Shamim Ahmed,1,1,3,3,Umran Shahid,9012,1
167,PL,3706,Accounts Officer,KHI,Others,Business 5,SSD,Noman K Dar,1,1,1,3,Syed Akram,9001,1
360,PL,567,Accounts Officer,KHI,Others,Business 5,Shared Services Finance,Shamim Ahmed,1,1,0,8,Mehjabeen Batlay,9001,1
1044,SL,2172,Accounts Officer,KHI,Aamir Jamil,Business 5,Shared Services Accounting,Faizan Ahmed(Shared Accounting),1,1,1,9,Yousaf Awan,9004,1
1257,EPL,2488,Accounts Officer,KHI,Others,Business 5,Shared Services Finance,Saba Ali,2,1,0,8,Umran Shahid,9001,4
1288,SL,2645,Accounts Officer,KHI,Shamim Ahmed,Business 5,Shared Services Finance,Saba Ali,2,1,0,9,Syed Akram,9001,1
1346,PL,2890,Accounts Officer,KHI,Others,Business 5,Shared Services Finance,Noman K Dar,1,1,1,8,Syed Akram,9001,5
1347,PL,2890,Accounts Officer,KHI,Others,Business 5,Shared Services Finance,Noman K Dar,1,1,1,8,Syed Akram,9001,5
1348,EPL,2890,Accounts Officer,KHI,Others,Business 5,Shared Services Finance,Noman K Dar,2,1,1,8,Syed Akram,9001,2
1370,PL,3496,Accounts Officer,KHI,Shamim Ahmed,Business 5,Shared Services Finance,Saba Ali,1,1,0,9,Naureen Rodrigues,9001,1


In [286]:
print(test['S_Month'].value_counts())
print(test['BUSINESS'].value_counts())

1    310
2    135
3      2
Name: S_Month, dtype: int64
Business 1    137
Business 6    108
Business 2     69
Business 3     65
Business 5     47
Business 7     13
Business 4      8
Name: BUSINESS, dtype: int64


In [287]:
month_breakdown = test.groupby('S_Month')[['TOTAL_NO_DAYS','pred']].sum()

In [288]:
# agregating by monthly predicitons 
month_breakdown['accuracy'] = 100*(month_breakdown['TOTAL_NO_DAYS'] - abs(month_breakdown['TOTAL_NO_DAYS']- month_breakdown['pred']))/month_breakdown['TOTAL_NO_DAYS']

In [289]:
month_breakdown

,TOTAL_NO_DAYS,pred,accuracy
S_Month,,,
1,732,803,90.300546
2,381,372,97.637795
3,21,7,33.333333


In [290]:
# agregating business wise leaves prediciton
business_breakdown = test.groupby('BUSINESS')[['TOTAL_NO_DAYS','pred']].sum()
business_breakdown['accuracy'] = 100*(business_breakdown['TOTAL_NO_DAYS'] - abs(business_breakdown['TOTAL_NO_DAYS']- business_breakdown['pred']))/business_breakdown['TOTAL_NO_DAYS']

In [291]:
business_breakdown

,TOTAL_NO_DAYS,pred,accuracy
BUSINESS,,,
Business 1,315,282,89.523810
Business 2,206,205,99.514563
Business 3,179,214,80.446927
Business 4,77,18,23.376623
Business 5,99,139,59.595960
Business 6,234,283,79.059829
Business 7,24,41,29.166667


In [292]:
# aggregatiing predictions department wise 
department_breakdown = test.groupby('DEPARTMENT')[['TOTAL_NO_DAYS','pred']].sum()
department_breakdown['accuracy'] = 100*(department_breakdown['TOTAL_NO_DAYS'] - abs(department_breakdown['TOTAL_NO_DAYS']- department_breakdown['pred']))/department_breakdown['TOTAL_NO_DAYS']

In [293]:
department_breakdown

,TOTAL_NO_DAYS,pred,accuracy
DEPARTMENT,,,
Administration,15,9,60.000000
Agriculture,56,47,83.928571
Animal Health,74,66,89.189189
Business 3,2,6,-100.000000
Chemical,1,2,0.000000
Chemical & Agri,4,5,75.000000
Cirin Pharma,50,58,84.000000
Commercial,10,23,-30.000000
Engineering Electrical And Instrumentation,105,79,75.238095


In [294]:
# aggregatiing predictions by leave types 

lt_breakdown = test.groupby('LEAVE_TYPE')[['TOTAL_NO_DAYS','pred']].sum()
lt_breakdown['accuracy'] = 100*(lt_breakdown['TOTAL_NO_DAYS'] - abs(lt_breakdown['TOTAL_NO_DAYS']- lt_breakdown['pred']))/lt_breakdown['TOTAL_NO_DAYS']
lt_breakdown

,TOTAL_NO_DAYS,pred,accuracy
LEAVE_TYPE,,,
EPL,30,32,93.333333
LR,19,9,47.368421
LW,35,42,80.000000
PAL,29,25,86.206897
PL,892,916,97.309417
SL,122,153,74.590164
TL,7,5,71.428571


In [295]:
# splitting all these breakdowns month wise

business = test.groupby(['S_Month','BUSINESS'])[['TOTAL_NO_DAYS','pred']].sum()

In [296]:
leave_type = test.groupby(['S_Month','LEAVE_TYPE'])[['TOTAL_NO_DAYS','pred']].sum()

In [297]:
dept = test.groupby(['S_Month','DEPARTMENT'])[['TOTAL_NO_DAYS','pred']].sum()

In [298]:
month = month_breakdown

In [299]:
business

TOTAL_NO_DAYS  pred
S_Month BUSINESS                       
1       Business 1            258   214
        Business 2             97   115
        Business 3            107   151
        Business 4             34     9
        Business 5             70   100
        Business 6            147   183
        Business 7             19    31
2       Business 1             57    68
        Business 2             88    83
        Business 3             72    63
        Business 4             43     9
        Business 5             29    39
        Business 6             87   100
        Business 7              5    10
3       Business 2             21     7

In [300]:
month['difference'] = abs(month['TOTAL_NO_DAYS']-month['pred'])
month


,TOTAL_NO_DAYS,pred,accuracy,difference
S_Month,,,,
1,732,803,90.300546,71
2,381,372,97.637795,9
3,21,7,33.333333,14


In [301]:
dept['difference'] = abs(dept['TOTAL_NO_DAYS']-dept['pred'])
dept

TOTAL_NO_DAYS  pred  \
S_Month DEPARTMENT                                                        
1       Administration                                          3     4   
        Agriculture                                            37    29   
        Animal Health                                          64    52   
        Business 3                                              2     6   
        Chemical & Agri                                         4     5   
        Cirin Pharma                                           32    31   
        Commercial                                              8    16   
        Engineering Electrical And Instrumentation             74    57   
        Finance                                                17    11   
        Head Office                                             1     6   
        Health & Safety                                        19     3   
        Human Resource                                          8     8   
        Information Technology                                  7    17   
        Marketing                                              11    25   
        Medical                                                23     4   
        Others                                                 94   124   
        Peads & Nutra                                           9     3   
        Pharmaceutical                                         12    13   
        Production                                            115   144   
        Raw Material                                            1     1   
        SSD                                                     1     1   
        Salary                                                  1     4   
        Sales                                                   0    11   
        Shared Services Accounting                             13    27   
        Shared Services Finance                                 3    18   
        Shipping                                                3     4   
        Supply Chain                                           53    37   
        Technical                                             102   108   
        Technical & Enginearing                                 9     5   
        Wyeth Pakistan                                          6    29   
2       Administration                                         12     5   
        Agriculture                                            19    18   
        Animal Health                                          10    14   
        Chemical                                                1     2   
        Cirin Pharma                                           18    27   
        Commercial                                              2     7   
        Engineering Electrical And Instrumentation             31    22   
        Finance                                                 7    14   
        Head Office                                             1     6   
        Human Resource                                         13     3   
        Information Technology                                 20    10   
        Marketing                                              76    31   
        Operations                                              1     8   
        Others                                                 52    34   
        Pharmaceutical                                         14    10   
        Power Generation                                        3     3   
        Production                                             50    53   
        Reporting Group                                         3     3   
        Salary                                                  1     4   
        Shared Services Finance                                 3    11   
        Shipping                                                0     2   
        Supply Chain                                           11    15   
  

In [305]:
leave_type.to_csv('C:\\Users\\binary\\Downloads\\leave_type_leaves.csv')